In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pint import UnitRegistry

import scipy as sp
from scipy.optimize import fsolve

from fluid import Fluid

import json

with open('turbine_conditions.json') as f:
    conditions = json.load(f)



In [2]:
ureg = UnitRegistry()
co2 = Fluid(fluid='co2', ureg=ureg)

Q_ = ureg.Quantity
m_out = Q_(0.26,  ureg.kilogram / ureg.second)
t_out = 150 + 273.15
delta_p = Q_(2.1 * 1e6,  ureg.pascal)

def strip(quantity):
    # check if quantity is a pint quantity type
    if isinstance(quantity, Q_):
        # if it is, return the magnitude
        return quantity.magnitude
    else:
        # otherwise return the value
        return quantity


In [3]:
invalid_solution = []

for i, condition in enumerate(conditions):
    
    for flow_1 in range(1,7):
        
        enthalpy1 = co2.enthalpy(P = condition[f'P{flow_1}'], T = condition[f'T{flow_1}'])
        
        for flow_2 in range(flow_1 + 1,7):
            
            enthalpy2 = co2.enthalpy(P = condition[f'P{flow_2}'], T = condition[f'T{flow_2}'])
            
            for exhaust in range(1,7):
                # test if solution is valid
                if (flow_1, flow_2, exhaust) in invalid_solution:
                    continue
                
                
                if exhaust == flow_1 or exhaust == flow_2:
                    invalid_solution.append([flow_1, flow_2, exhaust])
                    continue
                p_tot = Q_(condition[f'P{exhaust}'], ureg.pascal) + delta_p
                
                
                # check to make sure the output pressure is lower than both the inputs
                if strip(p_tot) > condition[f'P{flow_1}'] or strip(p_tot) > condition[f'P{flow_2}']:
                    invalid_solution.append([flow_1, flow_2, exhaust])
                    continue
                
                
                h_tot = co2.enthalpy(P = p_tot, T = t_out)
                
                #print(f"m_out={m_out}, h_tot={h_tot}, e1={enthalpy1}, e2={enthalpy2}")
                
                # confirm units and make dimensionless
                m_tot = strip(m_out)
                h_tot = strip(h_tot)
                h1 = strip(enthalpy1)
                h2 = strip(enthalpy2)
                
                # check that h_tot is between h1 and h2
                if not (h1 < h_tot < h2 or h2 < h_tot < h1):
                    invalid_solution.append([flow_1, flow_2, exhaust])
                    continue
                
                # solve the linear system
                A = np.array([[1, 1], [h1 - h_tot, h2 - h_tot]])
                b = np.array([m_tot, 0])
                m1, m2 = np.linalg.solve(A, b)

                
                # check that the mass fractions are positive
                if m1 < 0 or m2 < 0:
                    invalid_solution.append([flow_1, flow_2, exhaust])
                    continue
                print(f"{i} - {flow_1}, {flow_2}, {exhaust}:  {m1}, {m2}")
    
    
    
    

0 - 2, 4, 1:  0.04188691098795065, 0.21811308901204934
0 - 2, 4, 5:  0.04188691098795065, 0.21811308901204934
0 - 2, 4, 6:  0.04188691098795065, 0.21811308901204934
0 - 3, 4, 1:  0.0571006483812484, 0.20289935161875158
0 - 3, 4, 5:  0.0571006483812484, 0.20289935161875158
0 - 3, 4, 6:  0.0571006483812484, 0.20289935161875158
1 - 2, 4, 1:  0.043953261247471655, 0.21604673875252836
1 - 2, 4, 5:  0.043953261247471655, 0.21604673875252836
1 - 2, 4, 6:  0.043953261247471655, 0.21604673875252836
1 - 3, 4, 1:  0.058967051911084895, 0.20103294808891511
1 - 3, 4, 5:  0.058967051911084895, 0.20103294808891511
1 - 3, 4, 6:  0.058967051911084895, 0.20103294808891511
2 - 2, 4, 1:  0.04670931375855477, 0.21329068624144523
2 - 2, 4, 5:  0.04670931375855477, 0.21329068624144523
2 - 2, 4, 6:  0.04670931375855477, 0.21329068624144523
2 - 3, 4, 1:  0.06304884181281219, 0.1969511581871878
2 - 3, 4, 5:  0.06304884181281219, 0.1969511581871878
2 - 3, 4, 6:  0.06304884181281219, 0.1969511581871878
3 - 2, 4, 

In [4]:
valid_combinations = []

for condition in conditions:
    
    for flow_1 in range(1, 7):
        for flow_2 in range(flow_1 + 1, 7):
            for exhaust in range(1, 7):
                if [flow_1, flow_2, exhaust] in invalid_solution or [flow_1, flow_2, exhaust] in valid_combinations:
                    continue
                
                valid_combinations.append([flow_1, flow_2, exhaust])
    
print("Valid Combinations Include: ")
print(valid_combinations)


Valid Combinations Include: 
[[2, 4, 1], [2, 4, 5], [2, 4, 6]]


In [ ]:
for condition in conditions:
    enthalpy2 = co2.enthalpy(P = condition['P2'], T = condition['T2'])
    enthalpy4 = co2.enthalpy(P = condition['P4'], T = condition['T4'])
